In [ ]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import WikipediaAPIWrapper
from langchain.schema import SystemMessage
from langchain.document_loaders import WebBaseLoader
from langchain.tools import DuckDuckGoSearchResults
from langchain.tools import WikipediaQueryRun

# ============================================================
# 커스텀 도구를 정의합니다.


class DuckDuckGoSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")


class DuckDuckGoSearchTool(BaseTool):
    name: Type[str] = "DuckDuckGoSearchTool"
    description: Type[
        str
    ] = """
    Use this tool to perform web searches using the DuckDuckGo search engine.
    It takes a query as an argument.
    Example query: "Latest technology news"
    """
    args_schema: Type[DuckDuckGoSearchToolArgsSchema] = DuckDuckGoSearchToolArgsSchema

    def _run(self, query) -> Any:
        search = DuckDuckGoSearchResults()
        return search.run(query)


class WikipediaSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for on Wikipedia")


class WikipediaSearchTool(BaseTool):
    name: Type[str] = "WikipediaSearchTool"
    description: Type[
        str
    ] = """
    Use this tool to perform searches on Wikipedia.
    It takes a query as an argument.
    Example query: "Artificial Intelligence"
    """
    args_schema: Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query) -> Any:
        wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wiki.run(query)


class WebScrapingToolArgsSchema(BaseModel):
    url: str = Field(description="The URL of the website you want to scrape")


class WebScrapingTool(BaseTool):
    name: Type[str] = "WebScrapingTool"
    description: Type[
        str
    ] = """
    If you found the website link in DuckDuckGo,
    Use this to get the content of the link for my research.
    """
    args_schema: Type[WebScrapingToolArgsSchema] = WebScrapingToolArgsSchema

    def _run(self, url):
        loader = WebBaseLoader([url])
        docs = loader.load()
        text = "\n\n".join([doc.page_content for doc in docs])
        return text


class SaveToTXTToolArgsSchema(BaseModel):
    text: str = Field(description="The text you will save to a file.")


class SaveToTXTTool(BaseTool):
    name: Type[str] = "SaveToTXTTOOL"
    description: Type[
        str
    ] = """
    Use this tool to save the content as a .txt file.
    """
    args_schema: Type[SaveToTXTToolArgsSchema] = SaveToTXTToolArgsSchema

    def _run(self, text) -> Any:
        print(text)
        with open("research_results_ta.txt", "w") as file:
            file.write(text)
        return "Research results saved to research_results.txt"


# ============================================================
# 에이전트를 만듭니다.
# 이 때, 시스템 메세지를 만들어서 에이전트에게 전달합니다.

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
)

system_message = SystemMessage(
    content="""
        You are a research expert.

        Your task is to use Wikipedia or DuckDuckGo to gather comprehensive and accurate information about the query provided. 

        When you find a relevant website through DuckDuckGo, you must scrape the content from that website. Use this scraped content to thoroughly research and formulate a detailed answer to the question. 

        Combine information from Wikipedia, DuckDuckGo searches, and any relevant websites you find. Ensure that the final answer is well-organized and detailed, and include citations with links (URLs) for all sources used.

        Your research should be saved to a .txt file, and the content should match the detailed findings provided. Make sure to include all sources and relevant information.

        The information from Wikipedia must be included.

        Ensure that the final .txt file contains detailed information, all relevant sources, and citations.
        """
)

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=[
        DuckDuckGoSearchTool(),
        WikipediaSearchTool(),
        WebScrapingTool(),
        SaveToTXTTool(),
    ],
    agent_kwargs={"system_message": system_message},
)


# ============================================================
# 정상적으로 작동하는지 확인합니다.

query = "Research about the XZ backdoor"
results = agent.run(query)
print(results)

/var/folders/1y/n8ljkjp57xb_knsdd9hg0mf00000gn/T/ipykernel_32629/1931133571.py:101: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/var/folders/1y/n8ljkjp57xb_knsdd9hg0mf00000gn/T/ipykernel_32629/1931133571.py:124: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchai



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoSearchTool` with `{'query': 'XZ backdoor'}`




/Users/verobeach7/Developement/fullstack-gpt/env/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


snippet: What Does the Backdoor Do? Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. The backdoor manipulated sshd, the executable file used to make remote ..., title: The XZ Backdoor: Everything You Need to Know - WIRED, link: https://www.wired.com/story/xz-backdoor-everything-you-need-to-know/, snippet: Now that the XZ backdoor has control over sshd, attackers can possess the encryption key used to make the SSH connections hide their malicious code in the SSH login certificate and use it as an entry point to execute various malicious actions on the compromised device - such as stealing files or installing malware (e.g., ransomware ..., title: The XZ Backdoor explained - Cybernews, link: https://cybernews.com/editorial/xz-linux-backdoor-explained/, snippet: Because the backdoor was discovered before the malicious versions of xz Utils were added to production versions of Linux, "it's not really affecting anyone in the real world," Will 